<a href="https://colab.research.google.com/github/kevintabares777/Deep-Learning/blob/main/ModeloGPTNeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Descarga de Transformers
!pip install -U transformers datasets --quiet


In [2]:
#Importar librerias
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments


In [3]:
#Carga del DATASET
dataset = load_dataset("amazon_polarity")
small_dataset = dataset["train"].select(range(1000))  # Usamos solo 1000 ejemplos


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [4]:
#Modelo GPT-NEO
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-Neo no tiene pad_token, usamos eos_token

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [5]:
#TOKENS
def preprocess(batch):
    combined_texts = [
        f"Review Title: {title}\nReview Body: {content}"
        for title, content in zip(batch["title"], batch["content"])
    ]
    tokens = tokenizer(
        combined_texts,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokens["labels"] = tokens["input_ids"]
    return tokens

In [6]:
tokenized_dataset = small_dataset.map(preprocess, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [25]:
#ENTRENAMIENTO DEL MODELO
training_args = TrainingArguments(
    output_dir="./gptneo-amazon",
    per_device_train_batch_size=2,
    num_train_epochs=15,
    logging_steps=10,
    save_steps=500,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [26]:
trainer.train()

Step,Training Loss
10,1.260000
20,0.988000
30,1.433000
40,1.218400
50,1.407000
60,0.970500
70,1.096500
80,1.165100
90,1.442100
100,1.217500


TrainOutput(global_step=7500, training_loss=0.8768894537607829, metrics={'train_runtime': 2754.9632, 'train_samples_per_second': 5.445, 'train_steps_per_second': 2.722, 'total_flos': 3918106460160000.0, 'train_loss': 0.8768894537607829, 'epoch': 15.0})

In [27]:
#PRUEBA DEL MODELO
prompt = "Review Title: Great product\nReview Body:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
output = model.generate(input_ids, max_length=200, do_sample=True, temperature=0.7)

print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Review Title: Great product
Review Body: This is really good condition but the product does not waste your child, but it's not waste your time..
